In [ ]:
from conf import ONION_DIR
import os
import ast
import numpy as np

import pandas as pd
from conf import DATA_DIR

In [18]:
track_dataframes = []
for subdir, dirs, files in os.walk(ONION_DIR):
    for file in files:
        track_dataframe = pd.read_csv(os.path.join(subdir, file), sep='\t')
        track_dataframes += [track_dataframe]

In [19]:
onion_to_compare = pd.concat(track_dataframes)
onion_to_compare['gems_profile'] = onion_to_compare['gems_profile'].apply(lambda x: ast.literal_eval(x))
onion_to_compare = onion_to_compare.set_index('id')
onion_to_compare = pd.merge(onion_to_compare, onion_to_compare['gems_profile'].apply(pd.Series).fillna(value=0.), right_index=True, left_index=True).reset_index(drop=False)
onion_to_compare = onion_to_compare.drop(columns=['gems', 'genre', 'album_name', 'popularity', 'gems_profile', 'popularity class'])

In [20]:
rename_columns = {
    'id': 'song_code',
    'microgenre': 'genre',
    'song': 'title',
    'tenderness': 'tend',
    'peacefulness': 'peac',
    'joyful activation': 'joya',
    'wonder': 'wond',
    'power': 'ener',
    'nostalgia': 'nost',
    'tension': 'tens',
    'sadness': 'sadn',
    'transcendence': 'tran',
}
onion_to_compare = onion_to_compare.rename(columns=rename_columns)

In [74]:
inverse_rename_columns = {
    idx: label for label, idx in rename_columns.items()
}

In [21]:
rename_microgenres = {
    'trance': 'trance',
    'soft rock': 'rock',
    'soul': 'soul',
    'smooth jazz': 'jazz',
    'indie pop': 'pop',
    'swing': 'swing',
    'pop punk': 'punk',
    'dubstep': 'dubstep',
    'europop': 'europop',
    'doom metal': 'metal',
    'funk': 'funk',
    'downtempo': 'downtempo',
    'ballad': 'ballad',
    'disco': 'disco',
    'bossa nova': 'nova',
}
onion_to_compare.genre = onion_to_compare.genre.map(rename_microgenres)

In [22]:
onion_to_compare = onion_to_compare[['song_code', 'wond', 'tran', 'tend', 'nost', 'peac', 'joya', 'ener',
       'sadn', 'tens', 'title', 'artist', 'genre']]

In [23]:
onion_to_compare.to_csv(DATA_DIR + 'onion_to_compare.csv', index=False)

In [24]:
gems_9 = ['wond', 'tran', 'tend', 'nost', 'peac', 'joya', 'ener', 'sadn', 'tens']
f = onion_to_compare[gems_9].to_numpy().flatten().reshape(-1, 1)

In [25]:
(f > 0.).sum() / len(f)

0.34854059357370615

In [26]:
threshold = 0.
onion_bin_to_compare = onion_to_compare.copy()
onion_bin_to_compare[gems_9] = (onion_bin_to_compare[gems_9] > threshold) * 1.
onion_bin_to_compare.to_csv(DATA_DIR + 'onion_bin_to_compare.csv', index=False)

In [27]:
onion_bin_to_compare

,song_code,wond,tran,tend,nost,peac,joya,ener,sadn,tens,title,artist,genre
0,wDGNxY33wVBqikW2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,Not On Drugs (Ali Payami Remix),Tove Lo,disco
1,a8uQsr6gnzSCNrFB,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,Don't Let Me Be Misunderstood,Santa Esmeralda,disco
2,ZdoFOuRY0SRYxxuw,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,You Meant The World To Me,Taylor Dayne,disco
3,yQIJDbt3yfSoXAsU,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Scary But Fun,Grace Jones,disco
4,NFq9a3oLSVaGKJWx,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,I'm So Excited,The Pointer Sisters,disco
...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,wb4LtQFBXxV8SaZp,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,I Follow Rivers,Lykke Li,pop
449,tqOQJQT0ziGdhRqD,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,Video Games,Lana Del Rey,pop
450,yfm7RER1PWTqgjIp,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Royals,Lorde,pop
451,xKv9A6zDQOEySeNZ,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,Dog Days Are Over,Florence + the Machine,pop


Sparsity per emotion

In [28]:
f = onion_bin_to_compare[gems_9]
f.sum().sum() / len(f) * 9

28.231788079470196

In [29]:
onion_bin_to_compare

,song_code,wond,tran,tend,nost,peac,joya,ener,sadn,tens,title,artist,genre
0,wDGNxY33wVBqikW2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,Not On Drugs (Ali Payami Remix),Tove Lo,disco
1,a8uQsr6gnzSCNrFB,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,Don't Let Me Be Misunderstood,Santa Esmeralda,disco
2,ZdoFOuRY0SRYxxuw,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,You Meant The World To Me,Taylor Dayne,disco
3,yQIJDbt3yfSoXAsU,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Scary But Fun,Grace Jones,disco
4,NFq9a3oLSVaGKJWx,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,I'm So Excited,The Pointer Sisters,disco
...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,wb4LtQFBXxV8SaZp,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,I Follow Rivers,Lykke Li,pop
449,tqOQJQT0ziGdhRqD,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,Video Games,Lana Del Rey,pop
450,yfm7RER1PWTqgjIp,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,Royals,Lorde,pop
451,xKv9A6zDQOEySeNZ,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,Dog Days Are Over,Florence + the Machine,pop


In [33]:
onion_bin_to_compare.columns = ['item_id:token'] + list(onion_bin_to_compare.columns[1:])
onion_bin_to_compare = onion_bin_to_compare[list(onion_bin_to_compare.columns[0:10])]

In [70]:
index_to_gems_9 = {index: gems for index, gems in enumerate(gems_9)}

In [76]:
indices = [list(np.nonzero(x)[0].astype(int)) for x in onion_bin_to_compare[gems_9].to_numpy()]
emotions = [', '.join([inverse_rename_columns[index_to_gems_9[index]] for index in sub_indices]) for sub_indices in indices]

In [86]:
onion_bin_to_compare['emotions:token_seq'] = emotions

/tmp/ipykernel_25072/3262077964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onion_bin_to_compare['emotions:token_seq'] = emotions


In [88]:
onion_for_recbole = onion_bin_to_compare[['item_id:token', 'emotions:token_seq']]

I think neglecting the actual words is better

In [89]:
onion_for_recbole

,item_id:token,emotions:token_seq
0,wDGNxY33wVBqikW2,"tenderness, joyful activation"
1,a8uQsr6gnzSCNrFB,"tenderness, joyful activation, power"
2,ZdoFOuRY0SRYxxuw,"transcendence, tenderness"
3,yQIJDbt3yfSoXAsU,"transcendence, joyful activation"
4,NFq9a3oLSVaGKJWx,"tenderness, nostalgia, joyful activation, power"
...,...,...
448,wb4LtQFBXxV8SaZp,"transcendence, tenderness, nostalgia, joyful a..."
449,tqOQJQT0ziGdhRqD,"wonder, transcendence, tenderness, nostalgia, ..."
450,yfm7RER1PWTqgjIp,"wonder, transcendence, tenderness, peacefulnes..."
451,xKv9A6zDQOEySeNZ,"wonder, transcendence, tenderness, joyful acti..."


In [91]:
onion_bin_to_compare['emotions:token_seq'] = indices
onion_for_recbole = onion_bin_to_compare[['item_id:token', 'emotions:token_seq']]
onion_for_recbole

/tmp/ipykernel_25072/2953435376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onion_bin_to_compare['emotions:token_seq'] = indices


,item_id:token,emotions:token_seq
0,wDGNxY33wVBqikW2,"[2, 5]"
1,a8uQsr6gnzSCNrFB,"[2, 5, 6]"
2,ZdoFOuRY0SRYxxuw,"[1, 2]"
3,yQIJDbt3yfSoXAsU,"[1, 5]"
4,NFq9a3oLSVaGKJWx,"[2, 3, 5, 6]"
...,...,...
448,wb4LtQFBXxV8SaZp,"[1, 2, 3, 5, 7]"
449,tqOQJQT0ziGdhRqD,"[0, 1, 2, 3, 4, 7]"
450,yfm7RER1PWTqgjIp,"[0, 1, 2, 4, 5]"
451,xKv9A6zDQOEySeNZ,"[0, 1, 2, 5, 6]"


In [93]:
onion_for_recbole.to_csv('/home/marta/jku/emotion-popularity/notebooks/psyias2024/recbole_data/onion.item', sep='\t', index=None)